In [14]:
import os 
import json 
import pandas as pd 
import traceback

In [15]:
from langchain.chat_models import ChatOpenAI


In [16]:
from dotenv import load_dotenv
load_dotenv()#take environement variable from.env 

True

In [17]:
KEY=os.getenv("OPENAI_API_KEY")

In [19]:
llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo",temperature=0.5)

<ipython-input-19-832eca78438c>:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo",temperature=0.5)
